In [1]:
import time
import re
import requests
from requests_html import HTML

In [2]:
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
options = Options()
options.add_argument("--headless")

driver = webdriver.Chrome(options=options,service=Service(ChromeDriverManager().install()))

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\admin\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache


In [4]:
categories = [
    "https://www.amazon.com/gp/bestsellers/toys-and-games",
    "https://www.amazon.com/gp/bestsellers/electronics",
    "https://www.amazon.com/gp/bestsellers/books"
]

In [5]:
categories

['https://www.amazon.com/gp/bestsellers/toys-and-games',
 'https://www.amazon.com/gp/bestsellers/electronics',
 'https://www.amazon.com/gp/bestsellers/books']

In [6]:
first_url = categories[0]

In [7]:
driver.get(first_url)

In [8]:
body_el = driver.find_element(By.CSS_SELECTOR, "body")
html_str = body_el.get_attribute("innerHTML")

In [9]:
html_obj = HTML(html=html_str)

In [10]:
# new_links = [x for x in html_obj.links if x.startswith("/")]
# new_links = [x for x in new_links if "product-reviews/" not in x]
# page_links = [f"https://www.amazon.com{x}" for x in new_links]
page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
# page_links [x for x in page_links if extract_product_id_from_url(x) != None]

In [11]:
# page_links

In [12]:
def scrape_product_page(
    url, 
    title_lookup = "#productTitle",
    price_lookup = ".apexPriceToPay"
):
    driver.get(url)
    time.sleep(1.2)
    body_el = driver.find_element(By.CSS_SELECTOR, "body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [13]:
# first_product_link = page_links[0]
# first_product_link

In [14]:
# https://www.amazon.com/Max-Liquidator-6-Pack-Water-Blaster/dp/B0796JVBJ8/
# https://www.amazon.com/Play-Doh-Modeling-Compound-Non-Toxic-Exclusive/dp/B00JM5GW10/

# <base-url>/<slug>/dp/<product_id>/

In [15]:
# my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
# my_url = 'https://www.amazon.com/Max-Liquidator-6-Pack-Water-Blaster/dp/B0796JVBJ8/'

In [16]:
# regex = re.compile(my_regex_pattern)

In [17]:
# my_match = regex.match("abc")
# my_match = regex.match(my_url)
# print(my_match)
# my_match['product_id']

In [18]:
# my_match['slug']

In [19]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [20]:
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links

cleaned_links = clean_page_links(page_links)

In [21]:
len(page_links) == len(cleaned_links)

False

In [22]:
print("page_links", len(page_links))
print("cleaned_links", len(cleaned_links))

page_links 148
cleaned_links 29


In [23]:
cleaned_links

[{'url': 'https://www.amazon.com/PartySticks-Sticks-Party-Supplies-100pk/dp/B00N1QPNMA/ref=zg_bs_toys-and-games_8/130-8601364-7508027?pd_rd_i=B09Q7KP8GX&psc=1',
  'product_id': 'B00N1QPNMA'},
 {'url': 'https://www.amazon.com/Max-Liquidator-6-Pack-Water-Blaster/dp/B0796JVBJ8/ref=zg_bs_toys-and-games_15/130-8601364-7508027?pd_rd_i=B0796JVBJ8&psc=1',
  'product_id': 'B0796JVBJ8'},
 {'url': 'https://www.amazon.com/Taco-Cat-Goat-Cheese-Pizza/dp/B077Z1R28P/ref=zg_bs_toys-and-games_21/130-8601364-7508027?pd_rd_i=B077Z1R28P&psc=1',
  'product_id': 'B077Z1R28P'},
 {'url': 'https://www.amazon.com/Crayola-Colored-Pencil-Supplies-Assorted/dp/B00006RVTS/ref=zg_bs_toys-and-games_17/130-8601364-7508027?pd_rd_i=B09MH514FH&psc=1',
  'product_id': 'B00006RVTS'},
 {'url': 'https://www.amazon.com/LeapFrog-Animals-Book-Frustration-Packaging/dp/B085KTB1WY/ref=zg_bs_toys-and-games_3/130-8601364-7508027?pd_rd_i=B0953YFR2M&psc=1',
  'product_id': 'B085KTB1WY'},
 {'url': 'https://www.amazon.com/Free-Swimming-Ba

In [24]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            print(link, title, price)
        product_data = {
            "url": link,
            "product_id": product_id,
            "title": title,
            "price": price
        }
        data_extracted.append(product_data)
        
    return data_extracted

In [25]:
extracted_data = perform_scrape(cleaned_items=cleaned_links)

https://www.amazon.com/PartySticks-Sticks-Party-Supplies-100pk/dp/B00N1QPNMA/ref=zg_bs_toys-and-games_8/130-8601364-7508027?pd_rd_i=B09Q7KP8GX&psc=1 PartySticks Glow Sticks Party Supplies 100pk - 8 Inch Glow in the Dark Light Up Sticks Party Favors, Glow Party Decorations, Neon Party Glow Necklaces and Glow Bracelets with Connectors $11.95$11.95
https://www.amazon.com/Max-Liquidator-6-Pack-Water-Blaster/dp/B0796JVBJ8/ref=zg_bs_toys-and-games_15/130-8601364-7508027?pd_rd_i=B0796JVBJ8&psc=1 Max Liquidator 6-Pack Water Blaster Set $22.93$22.93
https://www.amazon.com/Taco-Cat-Goat-Cheese-Pizza/dp/B077Z1R28P/ref=zg_bs_toys-and-games_21/130-8601364-7508027?pd_rd_i=B077Z1R28P&psc=1 Taco Cat Goat Cheese Pizza $9.84$9.84
https://www.amazon.com/Crayola-Colored-Pencil-Supplies-Assorted/dp/B00006RVTS/ref=zg_bs_toys-and-games_17/130-8601364-7508027?pd_rd_i=B09MH514FH&psc=1 Crayola Colored Pencil Set, School Supplies, Assorted Colors, 36 Count, Long $5.97$5.97
https://www.amazon.com/LeapFrog-Animals

In [26]:
print(extracted_data)

[{'url': 'https://www.amazon.com/PartySticks-Sticks-Party-Supplies-100pk/dp/B00N1QPNMA/ref=zg_bs_toys-and-games_8/130-8601364-7508027?pd_rd_i=B09Q7KP8GX&psc=1', 'product_id': 'B00N1QPNMA', 'title': 'PartySticks Glow Sticks Party Supplies 100pk - 8 Inch Glow in the Dark Light Up Sticks Party Favors, Glow Party Decorations, Neon Party Glow Necklaces and Glow Bracelets with Connectors', 'price': '$11.95$11.95'}, {'url': 'https://www.amazon.com/Max-Liquidator-6-Pack-Water-Blaster/dp/B0796JVBJ8/ref=zg_bs_toys-and-games_15/130-8601364-7508027?pd_rd_i=B0796JVBJ8&psc=1', 'product_id': 'B0796JVBJ8', 'title': 'Max Liquidator 6-Pack Water Blaster Set', 'price': '$22.93$22.93'}, {'url': 'https://www.amazon.com/Taco-Cat-Goat-Cheese-Pizza/dp/B077Z1R28P/ref=zg_bs_toys-and-games_21/130-8601364-7508027?pd_rd_i=B077Z1R28P&psc=1', 'product_id': 'B077Z1R28P', 'title': 'Taco Cat Goat Cheese Pizza', 'price': '$9.84$9.84'}, {'url': 'https://www.amazon.com/Crayola-Colored-Pencil-Supplies-Assorted/dp/B00006RVT